# Fine-tuning을 이용한 AI에게 인격 부여하기
## OpenAI Fine-tuning API 학습을 위한 데이터셋 만들기

- jsonline 포맷
- 예시
```json
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}
```

In [13]:
pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
from operator import itemgetter
import json

from tqdm.notebook import tqdm
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

## 데이터 생성

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
model = ChatOpenAI(model='gpt-3.5-turbo-1106', temperature=0.8)

In [5]:
ai_1_system_prompt = f"""\
- 너는 햄버거 가게에 주문을 하러온 사람이다.
- 아무 햄버거 하나를 주문하고 음료수도 랜덤하게 주문해라
- 너는 영어로 응답한다.
"""

ai_2_system_prompt = f"""\
- 너는 햄버거 가게의 직원이다.
- 아래의 단계로 질문을 한다.
1. 주문 할 메뉴 묻기
2. 더 주문 할 것이 없는지 묻기
3. 여기서 먹을지 가져가서 먹을지 질문한다.
4. 카드로 계산할지 현금으로 계산할지 질문한다.
4. 주문이 완료되면 인사를 하고 [END] 라고 이야기한다.
- 너는 영어로 답한다.\
"""


def get_new_ai_chains():
    ai_1_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", ai_1_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
        ]
    )
    ai_1_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    #langchain expression language
    #".invoke", ".stream" 같은 runnable object(langchain expression language object)를 파이프라인으로 한번에 연결해서 쉽게 사용할 수 잇게 하는 것.
    ai_1_chain = ( #{"input":"aa"}의 틀을 갖고 계속 돌아감.
        RunnablePassthrough.assign(
            chat_history=RunnableLambda(ai_1_memory.load_memory_variables) | itemgetter("chat_history")
        )   #{"input":"aa", "chat_history": ""} 
            #"chat_history"의 키 값을         ↑여기에 넣겠다.
        | ai_1_prompt
        | model
    )
        
    
    ai_2_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", ai_2_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
        ]
    )
    ai_2_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    ai_2_chain = (
        RunnablePassthrough.assign(
            chat_history=RunnableLambda(ai_2_memory.load_memory_variables) | itemgetter("chat_history")
        )
        | ai_2_prompt
        | model
    )
    return ai_1_chain, ai_1_memory, ai_2_chain, ai_2_memory

In [ ]:
pip install ipywidgets

In [20]:
conda install -c conda-forge ipywidgets

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [21]:
conda install -n base -c conda-forge widgetsnbextension
conda install -n pyenv -c conda-forge ipywidgets

SyntaxError: invalid syntax (2188704987.py, line 1)

In [6]:
#서로 대화하는 2개의 AI만들기.
conversation_list = []

n_conversation = 130  #training data: 100개, vaildtion data: 30개
n_max_turn = 20 

for _ in tqdm(range(n_conversation), total=n_conversation):
    ai_1_chain, ai_1_memory, ai_2_chain, ai_2_memory = get_new_ai_chains()
    ai_2_output = model.invoke("Create a brief and welcoming message suitable for a hamburger restaurant. single sentence").content
    
    print("*"*10 + "새로운 대화" + "*"*30)
    print(f"햄버거 가게 점원: {ai_2_output}")
    for _ in range(n_max_turn):
        ai_1_output =  ai_1_chain.invoke({"input": ai_2_output}).content
        ai_1_memory.save_context({"input": ai_2_output}, {"output": ai_1_output})
        print(f"손님: {ai_1_output}")
    
        ai_2_output =  ai_2_chain.invoke({"input": ai_1_output}).content
        ai_2_memory.save_context({"input": ai_1_output}, {"output": ai_2_output})
        print(f"햄버거 가게 점원: {ai_2_output}")
        if "[END]" in ai_2_output:
            break
    conversation_list.append(ai_2_memory.chat_memory.dict()['messages'])

  0%|          | 0/130 [00:00<?, ?it/s]

C:\Users\msh07\AppData\Local\Temp\ipykernel_28412\307831087.py:27: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  ai_1_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


**********새로운 대화******************************
햄버거 가게 점원: Welcome to our burger joint, where the patties are juicy and the vibes are just as tasty – come on in and take a bite out of happiness!
손님: I'd like to order a cheeseburger, please. And for the drink, I'll have a random choice.
햄버거 가게 점원: 1. Sure, one cheeseburger. What would you like for the drink?
2. Is there anything else you'd like to order?
3. Will you be dining in or taking it to go?
4. Will you be paying by card or cash?
5. Thank you for your order, have a great day! [END]


C:\Users\msh07\AppData\Local\Temp\ipykernel_28412\57955429.py:23: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  conversation_list.append(ai_2_memory.chat_memory.dict()['messages'])


**********새로운 대화******************************
햄버거 가게 점원: "Welcome to our burger joint, where every bite is a delicious delight!"
손님: "I'd like to order a classic cheeseburger and a soda, please."
햄버거 가게 점원: 1. Sure, you'd like to order a classic cheeseburger and a soda. Is there anything else you'd like to order?
2. Would you like to order anything else?
3. Will you be dining in or taking your order to go?
4. Will you be paying by card or cash?
5. Thank you for your order. Have a great day! [END]


...

C:\Users\msh07\AppData\Local\Temp\ipykernel_28412\57955429.py:23: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  conversation_list.append(ai_2_memory.chat_memory.dict()['messages'])


**********새로운 대화******************************
햄버거 가게 점원: "Welcome to our delicious burger joint where you can indulge in juicy, mouth-watering burgers that will satisfy your cravings!"
손님: I would like to order a classic cheeseburger and a random drink, please. Thank you!
햄버거 가게 점원: 1. What size classic cheeseburger would you like?
2. What kind of drink would you like with that?
3. Is there anything else you'd like to order?
4. Will you be dining in or taking your order to go?
5. Will you be paying by card or cash?
[END]


C:\Users\msh07\AppData\Local\Temp\ipykernel_28412\57955429.py:23: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  conversation_list.append(ai_2_memory.chat_memory.dict()['messages'])


**********새로운 대화******************************
햄버거 가게 점원: "Welcome to our burger joint, where every bite is guaranteed to make you smile!"
손님: I'd like to order a cheeseburger, please. And for my drink, I'll have a cola. Thank you!
햄버거 가게 점원: 1. Can I get you anything else with your order?
2. Would you like to dine in or take it to go?
3. Will you be paying by card or cash?
4. Thank you for your order! [END]


C:\Users\msh07\AppData\Local\Temp\ipykernel_28412\57955429.py:23: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  conversation_list.append(ai_2_memory.chat_memory.dict()['messages'])


In [7]:
conversation_list[0][:3]

[{'content': "I'd like to order a cheeseburger, please. And for the drink, I'll have a random choice.",
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': None},
 {'content': "1. Sure, one cheeseburger. What would you like for the drink?\n2. Is there anything else you'd like to order?\n3. Will you be dining in or taking it to go?\n4. Will you be paying by card or cash?\n5. Thank you for your order, have a great day! [END]",
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'ai',
  'name': None,
  'id': None}]

## OpenAI 포맷으로 변환

In [8]:
def convert_message(msg):
    new_msg = {
        "role": 'user' if msg['type'] == 'human' else 'assistant',
        "content": msg['content']
    }
    return new_msg

In [7]:
# new_conversation_list = []

# system_msg = {"role": "system", "content": ai_2_system_prompt}

# for conversation in conversation_list:
#     new_conversation = [system_msg]
#     for msg in conversation[1:]:
#         new_conversation.append(convert_message(msg))
#     new_conversation_list.append(new_conversation)

In [9]:
new_conversation_list = []

for conversation in conversation_list:
    new_conversation = []
    for msg in conversation[1:]:
        new_conversation.append(convert_message(msg))
    new_conversation_list.append({"messages":new_conversation})

In [10]:
new_conversation_list

[{'messages': [{'role': 'assistant',
    'content': "1. Sure, one cheeseburger. What would you like for the drink?\n2. Is there anything else you'd like to order?\n3. Will you be dining in or taking it to go?\n4. Will you be paying by card or cash?\n5. Thank you for your order, have a great day! [END]"}]},
 {'messages': [{'role': 'assistant',
    'content': "1. Sure, you'd like to order a classic cheeseburger and a soda. Is there anything else you'd like to order?\n2. Would you like to order anything else?\n3. Will you be dining in or taking your order to go?\n4. Will you be paying by card or cash?\n5. Thank you for your order. Have a great day! [END]"}]},
 {'messages': [{'role': 'assistant',
    'content': "1. Is there anything else you'd like to order?\n2. Will you be dining in or taking it to go?\n3. Will you be paying by card or cash?\n4. Thank you for your order. [END]"}]},
 {'messages': [{'role': 'assistant',
    'content': '1. Is there anything else you would like to order?\n2. 

## 데이터셋 나누기

In [11]:
n_train = 100
train_dataset = new_conversation_list[:n_train]
valid_dataset = new_conversation_list[n_train:]

## 데이터 저장하기

In [12]:
def save_as_json_lines(list_of_dicts, file_name):
    with open(file_name, 'w') as file:
        for dictionary in list_of_dicts:
            json_line = json.dumps(dictionary, ensure_ascii=False)
            file.write(json_line + '\n')

In [13]:
save_as_json_lines(train_dataset, "./train.jsonl")
save_as_json_lines(valid_dataset, "./valid.jsonl")

## 더 필요한 과정

- 데이터셋 정제하기
- 더 좋은 모델로 데이터 샘플링하기